<a href="https://colab.research.google.com/github/lonerry/X-MAS_HACK/blob/main/christmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from bisect import bisect_right
from functools import lru_cache
from collections import defaultdict
from google.colab import files
import matplotlib.pyplot as plt
uploaded = files.upload()
ex_rates_dict = dict(zip(ex_rates['destination'], ex_rates['rate']))
base_currency = "USD"

def process_payments(providers_df, payments_df, output_filename):
    # Preprocess Providers DataFrame
    providers_df = providers_df.drop(columns=['LIMIT_BY_CARD'])
    providers_df['TIME'] = pd.to_datetime(providers_df['TIME'])
    providers_df['date'] = providers_df['TIME'].dt.date  # Extract date

    # Get first limits per ID and date
    first_limits = (providers_df.sort_values(by='TIME')
                    .groupby(['ID', 'date'])
                    .first()
                    .reset_index()[['ID', 'date', 'LIMIT_MIN', 'LIMIT_MAX']])
    providers_df = providers_df.merge(first_limits, on=['ID', 'date'], suffixes=('', '_first'))
    providers_df['LIMIT_MIN'] = providers_df['LIMIT_MIN_first']
    providers_df['LIMIT_MAX'] = providers_df['LIMIT_MAX_first']
    providers_df = providers_df.drop(columns=['LIMIT_MIN_first', 'LIMIT_MAX_first', 'date'])

    # Preprocess Payments DataFrame
    payments_df['eventTimeRes'] = pd.to_datetime(payments_df['eventTimeRes'], errors='coerce')
    payments_df['amount_base'] = payments_df.apply(
        lambda row: convert_to_base(row['amount'], row['cur'], ex_rates_dict, base_currency), axis=1
    )

    # Sort providers by TIME
    providers_df = providers_df.sort_values(by='TIME')

    # Create providers dictionary for quick state retrieval
    providers_dict = {}
    for pid, group in providers_df.groupby('ID'):
        times = group['TIME'].values
        state_cols = ['CONVERSION', 'AVG_TIME', 'MIN_SUM', 'MAX_SUM', 'LIMIT_MAX', 'LIMIT_MIN', 'COMMISSION', 'CURRENCY']
        states = group[state_cols].values
        providers_dict[pid] = (times, states, state_cols)

    unique_providers = list(providers_dict.keys())
    daily_usage = defaultdict(float)

    provider_stats = defaultdict(lambda: {'profit': 0.0, 'time': 0.0, 'success': 0, 'transactions': 0})

    @lru_cache(maxsize=None)
    def get_provider_state(pid, t):
        if pid not in providers_dict:
            return None
        times, states, state_cols = providers_dict[pid]
        idx = bisect_right(times, np.datetime64(t)) - 1
        if idx < 0:
            return None
        state = dict(zip(state_cols, states[idx]))
        return state

    @lru_cache(maxsize=None)
    def convert_to_usd(amount, currency):
        if currency == base_currency:
            return amount
        rate = ex_rates_dict.get(currency)
        if rate is None:
            raise ValueError(f"Exchange rate for {currency} not found.")
        return amount * rate

    def can_process_payment(payment, provider_info, daily_usage_key):
        amount_base = payment.amount_base
        amount = payment.amount
        payment_currency = payment.cur

        # 1. Check if provider state exists
        if provider_info is None:
            return False

        # 2. Check amount limits (in original currency)
        if not (provider_info['MIN_SUM'] <= amount <= provider_info['MAX_SUM']):
            return False

        # 3. Check currency match
        if payment_currency != provider_info['CURRENCY']:
            return False

        # 4. Check daily limit
        if daily_usage[daily_usage_key] + amount_base > provider_info['LIMIT_MAX']:
            return False

        return True

    flows = []

    for payment in payments_df.itertuples():
        payment_time = payment.eventTimeRes
        amount = payment.amount
        amount_base = payment.amount_base
        pay_date = payment_time.date()
        payment_currency = payment.cur
        candidates = []
        cumulative_time = 0  # Initialize cumulative processing time for the current transaction

        for pid in unique_providers:
            provider_state = get_provider_state(pid, payment_time)
            if provider_state is None:
                continue
            try:
                amount_usd = convert_to_usd(amount, payment_currency)
            except ValueError as e:
                print(e)
                continue
            expected_profit = (amount_usd - provider_state['COMMISSION'] * amount_usd) * provider_state['CONVERSION']
            score = expected_profit - 0.01 * provider_state['AVG_TIME']
            candidates.append((pid, provider_state, score))

        # Sort candidates by score in descending order
        candidates.sort(key=lambda x: x[2], reverse=True)
        attempt_flow = []
        success = False

        for candidate in candidates:
            pid, state, score = candidate
            provider_info = {
                'id': pid,
                'CONVERSION': state['CONVERSION'],
                'AVG_TIME': state['AVG_TIME'],
                'MIN_SUM': state['MIN_SUM'],
                'MAX_SUM': state['MAX_SUM'],
                'LIMIT_MAX': state['LIMIT_MAX'],
                'LIMIT_MIN': state['LIMIT_MIN'],
                'COMMISSION': state['COMMISSION'],
                'CURRENCY': state['CURRENCY']
            }

            daily_usage_key = (pid, pay_date)
            if can_process_payment(payment, provider_info, daily_usage_key):
                attempt_flow.append(str(pid))
                if cumulative_time + state['AVG_TIME'] > 60:
                    break
                cumulative_time += state['AVG_TIME']
                # Update provider stats
                success = True
        if success:
            provider_stats[pid]['profit'] += (amount_usd - state['COMMISSION'] * amount_usd) * state['CONVERSION']
            daily_usage[daily_usage_key] += amount_base
            provider_stats[pid]['time'] += cumulative_time
            provider_stats[pid]['success'] += 1
            flows.append('-'.join(attempt_flow))
        else:
            flows.append('no')

    # Assign flows to payments DataFrame
    payments_df['flow'] = flows

    # Compute penalties for not meeting minimum limits
    penalties = {}
    for (pid, date), usage in daily_usage.items():
        state = get_provider_state(pid, pd.Timestamp(date))
        if state:
            if usage < state['LIMIT_MIN']:
                penalty = 0.01 * state['LIMIT_MIN']  # 1% of LIMIT_MIN_USD
                penalties[(pid, date)] = penalty
                provider_stats[pid]['profit'] -= penalty

    # Save the processed payments to CSV
    payments_df = payments_df.drop(columns=['amount_base'])
    payments_df.to_csv(output_filename, index=False)

    # Calculate and display metrics
    total_penalties = sum(penalties.values())
    overall_profit_after_penalties = sum(stats['profit'] for stats in provider_stats.values())
    overall_profit_before_penalties = overall_profit_after_penalties + total_penalties

    total_success = sum(stats['success'] for stats in provider_stats.values())

    average_processing_time = (
        sum(stats['time'] for stats in provider_stats.values()) / total_success
        if total_success > 0 else 0
    )

    overall_conversion = (
        total_success / len(payments_df) if len(payments_df) > 0 else 0
    )
     # Построение графиков
    success_rates = {pid: stats['success'] for pid, stats in provider_stats.items()}
    avg_times = {pid: stats['time'] / stats['success'] if stats['success'] > 0 else 0 for pid, stats in provider_stats.items()}

    # # График успешности транзакций по провайдерам
    # plt.figure(figsize=(14, 8))
    # plt.bar(list(success_rates.keys()), list(success_rates.values()), color='blue', alpha=0.7)
    # plt.xlabel('Provider ID')
    # plt.ylabel('Number of Successful Transactions')
    # plt.title('Successful Transactions by Providers')
    # plt.xticks(np.arange(0,49, 1))
    # plt.show()

    # # Визуализация распределения времени обработки
    # plt.figure(figsize=(14, 8))
    # plt.bar(list(avg_times.keys()), list(avg_times.values()), color='green', alpha=0.7)
    # plt.xlabel('Provider ID')
    # plt.ylabel('Average Processing Time (seconds)')
    # plt.title('Average Processing Time by Providers')
    # plt.xticks(np.arange(0,49, 1))
    # plt.show()

    print(f"Results for {output_filename}:")
    print(f"Overall Profit After Penalties: {overall_profit_after_penalties:.2f} USD")
    print(f"Overall Conversion: {overall_conversion:.2%}")
    print(f"Average Processing Time: {average_processing_time:.2f} sec\n")

# Helper function to convert amount to base currency
def convert_to_base(amount, currency, ex_rates_dict, base_currency):
    if currency == base_currency:
        return amount
    else:
        rate = ex_rates_dict.get(currency)
        if rate is None:
            raise ValueError(f"Exchange rate for {currency} not found.")
        return amount * rate

# Load provider and payment datasets
providers_1 = pd.read_csv('/content/providers_1.csv')
providers_2 = pd.read_csv('/content/providers_2.csv')
payments_1 = pd.read_csv('/content/payments_1.csv')
payments_2 = pd.read_csv('/content/payments_2.csv')
ex_rates = pd.read_csv('/content/ex_rates.csv')

# Process each pair separately
process_payments(providers_1, payments_1, 'payments_result_with_failures_1.csv')
process_payments(providers_2, payments_2, 'payments_result_with_failures_2.csv')

# Download the resulting files
files.download('payments_result_with_failures_1.csv')
files.download('payments_result_with_failures_2.csv')
